In [0]:
CREATE OR REFRESH STREAMING TABLE bronzee_qualifying
COMMENT 'Raw customers data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS 
SELECT *,
       _metadata.file_path AS input_file_path,
       CURRENT_TIMESTAMP AS ingestion_timestamp
FROM cloud_files(
  '/Volumes/ishwariformulabox/schemalanding/volume2/qualifying.csv',
  'csv',
  map('cloudFiles.inferColumnTypes', 'true')
);

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_qualifying (
  CONSTRAINT valid_qualify_id EXPECT (qualifyId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_race_id EXPECT (raceId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_driver_id EXPECT (driverId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_constructor_id EXPECT (constructorId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_number EXPECT (number IS NOT NULL AND number > 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_position EXPECT (position IS NOT NULL AND position > 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_q1 EXPECT (q1 IS NOT NULL AND q1 REGEXP '^\\d{2}:\\d{2}\\.\\d$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_q2 EXPECT (q2 IS NOT NULL AND q2 REGEXP '^\\d{2}:\\d{2}\\.\\d$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_q3 EXPECT (q3 IS NOT NULL AND q3 REGEXP '^\\d{2}:\\d{2}\\.\\d$') ON VIOLATION DROP ROW
)
COMMENT 'Cleaned qualifying data'
TBLPROPERTIES ('quality' = 'silver')
AS
SELECT qualifyId,
       raceId,
       driverId,
       constructorId,
       number,
       position,
       q1,
       q2,
       q3
FROM STREAM(bronzee_qualifying);
